In [10]:
from Utils.loadset import getDataSet
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import tensorflow_probability as tfp
import os
physical_devices = tf.config.list_physical_devices('GPU') 
print("Num GPUs:", len(physical_devices)) 

from trainer import Trainer
try:
    from Utils.connection_cfg import *
except Exception as e:
    PSWD = None
    USRN = None
    
from Utils.Data import dataWrapper,provideData
from Utils.transform import cutOut

Num GPUs: 1


In [11]:
dimension = (128,128)
channels = 5
optimizer = Adam( lr = 1e-3 )
slices = [224,352,224,352]
slices_label = [32,64+32,32,64+32]
cutOutFrame = cutOut(slices)
cutOutFrame_label = cutOut(slices_label)

PRETRAINING_TRANSFORMATIONS = [cutOutFrame]
TRANSFORMATION = [cutOutFrame_label]
tfd = tfp.distributions
def NLL(y_true, y_hat):
    return -y_hat.log_prob(y_true[:,:])
                           
def mixd(output):
    rate = tf.math.exp(output[0,:,:,0]) #A 
    s = tf.math.sigmoid(output[0,:,:,1])
    components = [tfd.Deterministic(loc=tf.zeros_like(rate)), #E
     tfd.Poisson(rate=rate) #F 
     ]
    mixture = tfd.Mixture(
          cat=tfd.Categorical(probs=tf.stack([1-s, s],axis=-1)),#D
          components=components)
    
    return mixture
    
def testnetPoisson_2(input_shape,
           n_predictions=1,
           simpleclassification=None,
           flatten_output=False,
           activation_hidden="relu",
           activation_output="relu"):


    inputs = Input(shape=input_shape) 

    conv01 = Conv2D(10, kernel_size=(3, 3), padding="same")(inputs)       # 10 x 64x64
    conv01 = Activation(activation_hidden)(conv01)
    conv01_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv01)            # 10 x 32x32


    conv02 = Conv2D(20, kernel_size=(3, 3), padding="same")(conv01_pool)  # 20 x 32x32
    conv02 = Activation(activation_hidden)(conv02)
    conv02_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv02)            # 20 x 16x16


    conv03 = Conv2D(20, kernel_size=(3, 3), padding="same")(conv02_pool)  # 20 x 16x16
    conv03 = Activation(activation_hidden)(conv03)
    conv03_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv03)            # 20 x 8x8


    conv04 = Conv2D(20, kernel_size=(3, 3), padding="same")(conv03_pool)  # 20 x 8x8
    conv04 = Activation(activation_hidden)(conv04)
    conv04_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv04)            # 20 x 4x4


    ### UPSAMPLING:
    up04 = UpSampling2D((2, 2))(conv04_pool)    # 20 x 8x8
    up04 = concatenate([conv04, up04], axis=3)  # 20+20 x 8x8


    up03 = UpSampling2D((2, 2))(up04)           # 40 x 16x16
    up03 = concatenate([conv03, up03], axis=3)  # 20+40 x 16x16


    up02 = UpSampling2D((2, 2))(up03)           # 60 x 32x32
    up02 = concatenate([conv02, up02], axis=3)  # 20+60 x 32x32


    up01 = UpSampling2D((2, 2))(up02)           # 80 x 64x64
    up01 = concatenate([conv01, up01], axis=3)  # 10+80 x 64x64
    up01 = MaxPooling2D((2, 2), strides=(2, 2))(up01)
    
    output = Conv2D(7, (1, 1), activation="selu")(up01)  # 1 x 64x64
    
    output = Flatten()(output)
    output = Dense(16*16)(output)
    output = Dropout(0.1)(output)
    output = Dense(64*64)(output)
    #output = Activation(tf.math.softplus)(output)
    
    output = tfp.layers.IndependentPoisson((64,64,1))(output)
    model = Model(inputs=inputs, outputs=output)
    return model

In [15]:
batch_size = 100
years = [2015,2016,2017]
train, test = provideData(dimension=dimension,
                          batch_size=batch_size,
                          channels = channels,
                          timeToPred = 5,
                          year = years,
                          transform=TRANSFORMATION,
                          preTransformation=PRETRAINING_TRANSFORMATIONS)

t = Trainer(testnetPoisson_2,
                    NLL,
                    (train,test),
                    batch_size = batch_size,
                    optimizer=optimizer,
                    dimension = dimension,
                    channels = channels,
                    metrics = ["mse","mae"])

from Utils.Data import getListOfFiles
model_h5 = getListOfFiles("./model_data/testnetPoisson_2_function")
#find best model

#loss
#for i in model_h5:
    


Found Year :  2015 => won't download this year again... please check for consistency
Found Year :  2016 => won't download this year again... please check for consistency
Found Year :  2017 => won't download this year again... please check for consistency
Finished Loading Dataset
 
[DEBUG] Data
[DEBUG] time to predict:  5
[DEBUG] Data
[DEBUG] time to predict:  5
[Loaded file]  ./model_data/testnetPoisson_2_function/testnetPoisson_2_function128x128x5.h5
5
[Loaded file]  ./model_data/testnetPoisson_2_function/testnetPoisson_2_function128x128x5history.json
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 5) 0                                            
__________________________________________________________________________________________________
conv2d_5 (Conv2D)           

In [28]:
#model = t.model
import re

regex = r"model-([0-9]+)-([0-9\.]+)-([0-9\.]+)\.h5"
model_chkpts = []
for chkpt in model_h5:
    matches = re.search(regex,chkpt)
    if matches:
        epoch = int(matches[1])
        loss  = float(matches[2])
        val_loss = float(matches[3])
        model_chkpts.append((epoch,val_loss,loss,chkpt))
        
sorted_by_epochs   = sorted(model_chkpts, key=lambda tup: tup[0])
sorted_by_loss     = sorted(model_chkpts, key=lambda tup: tup[1])
sorted_by_val_loss = sorted(model_chkpts, key=lambda tup: tup[2])


for i in range(len(sorted_by_epochs)):
    print(sorted_by_epochs[i][0],sorted_by_loss[i][1],sorted_by_val_loss[i][2])
    
    
model.load_weights(sorted_by_val_loss[i][3])

1 42.024609 33.842682
1 42.114346 35.257973
2 42.35955 35.362888
2 42.443684 35.524509
3 42.68811 35.695362
3 43.071289 35.894077
4 43.702473 36.219025
5 46.304604 37.147449
10 48.068161 39.229645
11 51.364555 40.127987
12 52.095764 40.952053
13 56.427193 44.18272
15 63.299557 45.091984
16 83.471931 46.712036
17 85.924782 49.05825
18 88.382278 69.456985
31 124.336014 124.673431
34 5394.1875 2177.331543
35 5428.422363 2224.683105
36 5549.835449 2282.901367
40 5673.18457 2360.178223
41 6108.28418 2389.879639
44 6483.583984 2421.743164
48 6811.31543 2475.360352
51 6947.900391 2530.189697
52 7044.596191 2607.694092
53 7309.921387 2691.209717
64 7415.59082 2820.583008
71 9350.608398 2988.757812
72 9435.573242 3253.788086
73 10868.753906 3352.783203
76 11628.861328 3487.139648
91 13259.867188 3686.416016
92 18285.294922 3719.620605
93 20257.445312 3775.344482
94 23629.240234 7556.141113
95 26630.080078 8772.947266
96 29225.294922 17252.259766
98 38334.238281 132333.65625
99 182807.703125 251

ValueError: You are trying to load a weight file containing 6 layers into a model with 7 layers.

In [3]:
dimension = (64,64)
batch_size = 10
channels = 5
optimizer = Adam( lr = 1e-6 )
slices = [256,320,256,320]
cutOutFrame = cutOut(slices)

categorical_list = [0,1,5,10,15,30,60,120]
categorical = ToCategorical(categorical_list)

PRETRAINING_TRANSFORMATIONS = [cutOutFrame]
TRANSFORMATIONS = [categorical]
TRANSFORMATIONS = None

def NLL(y_true, y_hat):
    return -y_hat.log_prob(y_true)


def provideData(flatten=False,dimension=dimension,batch_size=60,transform=None,preTransformation=None):

    getDataSet(DatasetFolder,year=[2017],username=USRN,pswd=PSWD)
    train,test = dataWrapper(PathToData,
                            dimension=dimension,
                            channels=channels,
                            batch_size=batch_size,
                            overwritecsv=True,
                            flatten=flatten,
                            onlyUseYears=[2017],
                            transform=transform,
                            preTransformation=preTransformation)
    
    return train,test
DatasetFolder = "./Data/RAW"
PathToData = os.path.join(DatasetFolder,"MonthPNGData")

train, test = provideData(dimension=dimension,
                          batch_size=batch_size,
                          transform=TRANSFORMATIONS,
                          preTransformation=PRETRAINING_TRANSFORMATIONS)

Found Year :  2017 => won't download this year again... please check for consistency
Finished Loading Dataset
 


In [11]:
y = train[0][1]
print(y.shape)
cat = tfp.distributions.Categorical(probs=tf.zeros_like(tf.expand_dims(y,axis=-1) ))
print(tf.stack([1-y[:,:,:,0], y[:,:,:,0]],axis=-1).shape)
cat = tfp.distributions.Categorical(probs=tf.stack([1-y, y],axis=-1))
pos = tfp.distributions.Poisson(rate=y)
det = tfp.distributions.Deterministic(loc=y)

print(cat)
print(det)
print(pos)
components = [det,pos]
mix = tfd.Mixture(cat = cat,components=components)
print(mix)

(10, 64, 64, 1)
(10, 64, 64, 2)
tfp.distributions.Categorical("Categorical", batch_shape=[10, 64, 64, 1], event_shape=[], dtype=int32)
tfp.distributions.Deterministic("Deterministic", batch_shape=[10, 64, 64, 1], event_shape=[], dtype=float64)
tfp.distributions.Poisson("Poisson", batch_shape=[10, 64, 64, 1], event_shape=[], dtype=float64)
tfp.distributions.Mixture("Mixture", batch_shape=[10, 64, 64, 1], event_shape=[], dtype=float64)


In [16]:
print(cat)
print(pos)
print(det)

tfp.distributions.Categorical("Categorical", batch_shape=[1, 64, 64], event_shape=[], dtype=int32)
tfp.distributions.Poisson("Poisson", batch_shape=[1, 64, 64, 1], event_shape=[], dtype=float64)
tfp.distributions.Deterministic("Deterministic", batch_shape=[1, 64, 64, 1], event_shape=[], dtype=float64)
